This notebook was created by Donna Faith Go.

In [1]:
# standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

# beaultifulsoup
import requests
from bs4 import BeautifulSoup

# selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time

## List of Indices

In [2]:
# setting up driver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# access website
url = "https://english.sse.com.cn/markets/indices/data/list/constituents/index.shtml?COMPANY_CODE=000001&INDEX_Code=000001"
driver.get(url)

# get table
wait = WebDriverWait(driver, 10)
table = wait.until(
    EC.presence_of_element_located((By.CLASS_NAME, "js_constituentsList"))
)

In [3]:
# get companies and their stock numbers
companies = []
page_num = 1

while True:
    try:
        print(f"Scraping page {page_num}...")
        
        # wait for table to load
        wait.until(
            EC.presence_of_element_located((By.CLASS_NAME, "js_constituentsList"))
        )
        
        # beautiful soup for current page
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        rows = soup.find('tbody').find_all('tr')
        
        # data from current page
        for row in rows:
            cols = row.find_all('td')
            if len(cols) >= 2:
                company = {
                    "Company Name": cols[0].find('a').get_text(), 
                    "Company Code": cols[1].get_text()
                }
                companies.append(company)
        
        # next button
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "a.flip[title='Next Page']")            
            if "disabled" in next_button.get_attribute("class") or not next_button.is_enabled():
                print("Reached the last page. Stopping.")
                break
                
            # click button
            driver.execute_script("arguments[0].click();", next_button)            
            time.sleep(2)
            
            page_num += 1
            
        except NoSuchElementException:
            print("No more pages available. Scraping complete.")
            break
            
    except Exception as e:
        print(f"Error on page {page_num}: {e}")
        break

driver.quit()

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...
Scraping page 18...
Scraping page 19...
Scraping page 20...
Scraping page 21...
Scraping page 22...
Scraping page 23...
Scraping page 24...
Scraping page 25...
Scraping page 26...
Scraping page 27...
Scraping page 28...
Scraping page 29...
Scraping page 30...
Scraping page 31...
Scraping page 32...
Scraping page 33...
Scraping page 34...
Scraping page 35...
Scraping page 36...
Scraping page 37...
Scraping page 38...
Scraping page 39...
Scraping page 40...
Scraping page 41...
Scraping page 42...
Scraping page 43...
Scraping page 44...
Scraping page 45...
Scraping page 46...
Scraping page 47...
Scraping page 48...
Scraping page 49...
Error on page 49: name 'NoSuc

The code above takes around 3-5 minutes to run.

In [4]:
# Save the data
companies_df = pd.DataFrame(companies)
companies_df.to_pickle('SSE companies list.pkl')
# companies_df.to_csv('SSE companies list.csv')
print(f"Scraped {len(companies)} companies from {page_num} pages")

Scraped 2444 companies from 49 pages


In [5]:
filepath = 'SSE companies list.pkl'
with open(filepath, 'rb') as f:
    companies_df = pickle.load(f)
companies_df.head()

,Company Name,Company Code
0,"SHANGHAI PUDONG DEVELOPMENT BANK CO., LTD.",600000
1,"Guangzhou Baiyun International Airport Co.,ltd.",600004
2,"Dongfeng Automobile Co.,LTD",600006
3,China World Trade Center Company Ltd.,600007
4,Beijing Capital Eco-Environment Protection Gro...,600008
